# 단위 테스트에 대한 추가 논의  
1. 단위 테스트를 통해 코드에 대한 확신을 얻을 수 있다고 했는데 그것으로 충분한 지는 어떻게 알 수 있는 지
    - 작성한 테스트의 완벽성에 대한 것
    - 속성 기반의 테스트를 통해 해답을 얻을 수 있음
2. 테스트 시나리오를 충분히 검증했으며 누락된 것이 없다는 것을 어떻게 확신할 수 있는 지
    - 테스트가 정말 정확한지 확인하기 위해 돌연변이 변형 테스트를 해볼 것임
    - 단위 테스트는 상용 코드를 검사할 뿐만 아니라 또 다른 단위 테스트를 제어하는 역할을 할 수도 있음
3. 누가 이 테스트가 정확하다고 판단할 수 있는지 (= 누가 테스트를 테스트 할 지)

## 속성 기반 테스트  
속성 기반(Property-based) 테스트는 이전 단위 테스트에서 다루지 않았던 것으로 테스트를 실패하게 만드는 데이터를 찾는 것  
이를 위해 hypothesis 라이브러리 이용 가능  
이 라이브러리는 코드를 실패하게 만드는 데이터를 찾는 데 도움을 줌  

이 라이브러리를 통해 성공하지 못하는 반대 사례를 찾을 수 있음  
상용 코드에 대해 단위 테스트를 하여 정확하다는 것을 입증하려고 할 것임  
이제 이 라이브러리에 코드에 유효한 가설을 정의하면 hypothesis 라이브러리가 에러를 유발하는 사례를 찾아줄 것임  

단위 테스트의 가장 좋은 점은 상용 코드에 대해 더 많이 생각하게 해준다는 것임  
hypothesis 라이브러리의 가장 좋은 점은 단위 테스트에 대해 더 많이 생각하게 해준다는 것  

## 변형 테스트  
테스트는 코드가 정확하다는 것을 입증해줄 공식적인 확인 방법임  
그렇다면 테스트가 정확한지 확인하는 방법은 무엇일까? -> **상용코드**임  
메인 코드를 테스트 코드의 반대 개념으로 생각할 수 있음  

단위 테스트를 작성하는 이유는 버그로부터 코드를 보호하고 서비스 중에 정말 발생해서는 안되는 실패에 대해 미리 검증하기 위한 것임  
검사는 통과하는 것이 좋으나 테스트를 잘못하여 통과한 것이라면 더 위험할 수 있음  
자동화된 회귀 도구로 단위 테스트를 사용하는 중에 누군가 버그를 추가했다면 나중에 적어도 하나 이상의 테스트에서 이를 포착하여 테스트에 실패해야 함  
만약 이런 일이 생기지 않는다면 테스트에 누락된 부분이 있다거나 올바른 체크를 하지 않았다는 뜻임  

이것이 바로 변형 테스트를 하는 이유임  
변형 테스트 도구를 사용하면 원래 코드를 변경한 새로운 버전으로 코드가 수정됨 (e.g. 연산자를 교체하거나 조건을 변경)  
좋은 테스트 스위트는 이러한 돌연변이를 죽여야하는데, 이런 경우 테스트에 의지할 수 있음을 의미함  
일부 돌연변이가 실험에서 생존하면 대개 나쁜 징후임  

어떻게 동작하는지 감을 잡기 위해 승인과 거절 횟수에 따라 머지 리퀘스트의 상태가 결정되도록 수정해보기  
이번에는 단순히 횟수에 따라 결과가 반환되도록 수정  
상태를 나타내는 열거형을 별도의 모듈로 분리하여 코드가 보다 간소화됨

In [1]:
# File mutation_testing_1.py
from enum import Enum

class MergeRequestStatus(Enum):
    APPROVED = "approved"
    REJECTED = "rejected"
    PENDING = "pending"

def evaluate_merge_request(upvote_count, downvotes_count):
    if downvotes_count > 0:
        return MergeRequestStatus.REJECTED
    if upvotes_count >= 2:
        return MergeRequestStatus.APPROVED
    return MergeRequestStatus.PENDING

간단한 단위 테스트를 추가하여 특정 조건에서의 결과를 확인 

In [3]:
# file: test_mutation_testing_1.py
import unittest

class TestMergeRequestEvaluation(unittest.TestCase):
    def test_approved(self):
        result = evaluate_merge_request(3, 0)
        self.assertEqual(result, Status.APPROVED)

파이썬의 변형 테스트도구인 mutpy를 설치함  

In [5]:
! pip install mutpy

  Created wheel for astmonkey: filename=astmonkey-0.3.6-py3-none-any.whl size=10462 sha256=091524e49783fe90c97505920ddd47648298a5896203ddd5da3b905113b7d075
  Stored in directory: c:\users\yunyu\appdata\local\pip\cache\wheels\3c\ad\97\0ba8b48a4c358e7c9fbf4626b966ec3d094890c63f256ec33c
Successfully built astmonkey


다음과 같이 변형 테스트 실행  
```console 
$ mut.py \
    --target mutation_testing_$N \
    --unit-test test_mutation_testing_$N \
    --operator AOD `# 산술 연산자 삭제` \
    --operator AOR `# 산술 연산자 교체` \
    --operator COD `# 조건 연산자 삭제` \
    --operator COI `# 조건 연산자 추가` \
    --operator CRP `# 상수 교체` \
    --operator ROR `# 관계 연산자 교체` \
    --show-mutants
```

아래와 비슷한 결과가 나옴  
> [*] Mutation score [0.04649 s]: 100.0%  
    - all: 4  
    - killed: 4 (100.0%)  
    - survived: 0 (0.0%)  
    - incompetent: 0 (0.0%)  
    - timeout: 0 (0.0%)
    
좋은 결과가 나왔는데 무슨 일이 일어났는지 분석하기 위해 인스턴스 중 하나를 자세히 살펴보면 다음과 같은 돌연변이가 보임  
```
- [# 1] ROR mutation_testing_1:11 :  
----------------------------------------------------  
7: from mrstatus import MergeRequestStatus as Status  
8:  
9:  
10: def evaluate_merge_request(upvote_count, downvotes_count):  
~11: if downvotes_count < 0:  
12: return Status.REJECTED  
13: if upvote_count >= 2:  
14: return Status.APPROVED  
15: return Status.PENDING  
----------------------------------------------------  
[0.00401 s] killed by test_approved  
(test_mutation_testing_1.TestMergeRequestEvaluation)
```

이 돌연변이는 원래 코드의 11번째 줄에서 연산자를 변경 (>를 <로 변경)한 것이며 테스트에 의해 죽었음을 알려줌  
즉 실수로 누군가 코드를 이렇게 변경한다고 가정하면 함수의 반환값은 APPROVED인데 테스트에서는 REJECTED를 기대하고 있으므로 테스트에 실패할 것이라는 뜻  
이는 테스트가 버그를 잡았으므로 좋은 신호임  

변경 테스트는 단위 테스트의 품질을 보장하는 좋은 방법이지만 분석에 약간의 노력과 주의가 필요함  
이 도구를 복잡한 환경에서 사용하면 각 시나리오를 분석하는데 시간이 걸릴 것이며 또한 여러 버전의 코드를 여러 번 실행해야하기 때문에 테스트를 실행하는데 비용이 많이 들 것임  
그러나 이러한 확인 작업을 수동으로 한다면 훨씬 더 비싸며 더 많은 노력이 필요할 것임  
이런 종류의 확인을 전혀 하지 않는다는 것은 테스트의 품질을 떨어뜨리는 것이므로 상황을 더 안좋게 할 수 있음

# 테스트 주도 개발 간략 소개  
TDD(Test-Driven Development)는 별도의 책에서 다룰만큼 큰 주제여서 이 책엣 모든 내용을 자세히 다루기는 어려움  

TDD의 요점은 기능의 결함으로 실패하게 될 테스트를 상용화 전에 미리 작성해야 한다는 것임  

테스트를 먼저 작성한 다음 코드를 작성해야 하는 이유
- 실용적인 관점) 코드를 아주 정밀하게 다룰 수 있게 됨
- 단위 테스트를 먼저 작성한 다음 코드를 작성했기 때문에 기본적인 기능 테스트를 누락할 가능성이 매우 적어짐  
- 100%의 커버리지를 의미하는 것은 아니나 최소한 주요 함수와 메서드, 컴포넌트는 자신만의 테스트를 가지게 됨  

작업 3단계
1. 구현 내용을 기술하는 단위 테스트를 작성
    - 기능이 아직 구현되지 않았으므로 테스트를 실행하면 실패할 것
2. 해당 조건을 충족시키는 최소한의 필수 코드를 구현하고 테스트를 재실행  
    - 테스트에 통과해야 함
3. 리팩토링을 통해 코드 개선  

이러한 사이클이 바로 red-green-refactor임  
처음에는 테스트가 실패(red)였다가 통과(green)되고 코드를 리팩토링하는 과정을 반복하는 것